In [1]:
from bertopic import BERTopic
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from tqdm import tqdm
from umap import UMAP
from hdbscan import HDBSCAN
import hdbscan
from bertopic.vectorizers import ClassTfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from bertopic.representation import PartOfSpeech
import os
import json
import openml
import spacy

/Users/ivang/miniconda3/envs/mvp/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from dotenv import load_dotenv

load_dotenv()

True

# Data preparation

Download datasets

In [3]:
# datasets_list = openml.datasets.list_datasets()
# ids = list(datasets_list.keys())
# print("Number of datasets: ", len(ids))

# datasets = []
# # # Removing non-dataset entries 
# # ids.remove(4537)
# # ids.remove(4546)
# # ids.remove(4562)

# for dataset_id in tqdm(ids, desc="Downloading datasets"):
#     try:
#         dataset = openml.datasets.get_dataset(dataset_id, download_features_meta_data=True)
#     except openml.exceptions.OpenMLServerException:
#         # If the first attempt fails, try again without features metadata
#         dataset = openml.datasets.get_dataset(dataset_id, download_features_meta_data=False)
#     datasets.append(dataset)

# datasets[0]

In [4]:
# df = pd.DataFrame([dataset.__dict__ for dataset in datasets])
# # Add the OpenML URL to the dataframe
# df['openml_url'] = [dataset.openml_url for dataset in datasets]
# df['upload_date'] = [dataset.upload_date for dataset in datasets]
# # All datasets which have None description, make it an empty string ""
# df['description'] = df['description'].fillna('')
# df

Save dataframe, so datasets aren't refetched every time.

In [5]:
# df.to_pickle("datasets_checkpoint.pkl")

Load datasets from the saved CSV

In [6]:
df = pd.read_pickle("datasets_checkpoint.pkl")
df

,ignore_attribute,dataset_id,name,version,description,cache_format,format,creator,contributor,collection_date,...,parquet_file,_dataset,_parquet_url,_features,_qualities,_no_qualities_found,data_pickle_file,data_feather_file,feather_attribute_file,openml_url
0,None,2,anneal,1,**Author**: Unknown. Donated by David Sterling...,pickle,ARFF,"[David Sterling, Wray Buntine]",David Sterling and Wray Buntine,1990,...,None,None,http://openml1.win.tue.nl/dataset2/dataset_2.pq,"{0: [0 - family (nominal)], 1: [1 - product-ty...",None,False,None,None,None,https://www.openml.org/d/2
1,None,3,kr-vs-kp,1,Author: Alen Shapiro\nSource: [UCI](https://ar...,pickle,ARFF,Alen Shapiro,Rob Holte,1989-08-01,...,None,None,http://openml1.win.tue.nl/dataset3/dataset_3.pq,"{0: [0 - bkblk (nominal)], 1: [1 - bknwy (nomi...",None,False,None,None,None,https://www.openml.org/d/3
2,None,4,labor,1,**Author**: Unknown\n**Source**: Collective Ba...,pickle,ARFF,Collective Bargaining Review of Labour Canada,Stan Matwin,1988-11-01,...,None,None,http://openml1.win.tue.nl/dataset4/dataset_4.pq,"{0: [0 - duration (numeric)], 1: [1 - wage-inc...",None,False,None,None,None,https://www.openml.org/d/4
3,None,5,arrhythmia,1,"**Author**: H. Altay Guvenir, Burak Acar, Hald...",pickle,ARFF,"[H. Altay Guvenir, Burak Acar, Haldun Muderris...",None,1998-01-01,...,None,None,http://openml1.win.tue.nl/dataset5/dataset_5.pq,"{0: [0 - age (numeric)], 1: [1 - sex (nominal)...",None,False,None,None,None,https://www.openml.org/d/5
4,None,6,letter,1,**Author**: David J. Slate \n**Source**: [UCI...,pickle,ARFF,David J. Slate,None,1991-01-01,...,None,None,http://openml1.win.tue.nl/dataset6/dataset_6.pq,"{0: [0 - x-box (numeric)], 1: [1 - y-box (nume...",None,False,None,None,None,https://www.openml.org/d/6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5809,None,46462,pollution,5,air quality,pickle,arff,None,None,None,...,None,None,https://openml1.win.tue.nl/datasets/0004/46462...,"{0: [0 - ID (numeric)], 1: [1 - log_pSat_Pa (n...",None,False,None,None,None,https://www.openml.org/d/46462
5810,None,46463,pollution,6,air quality,pickle,arff,None,None,None,...,None,None,https://openml1.win.tue.nl/datasets/0004/46463...,"{0: [0 - ID (numeric)], 1: [1 - log_pSat_Pa (n...",None,False,None,None,None,https://www.openml.org/d/46463
5811,None,46467,IBRD_Loans_Classification,4,The International Bank for Reconstruction and ...,pickle,arff,Anna Wiewer,OpenML Community,2024-12-01,...,None,None,https://openml1.win.tue.nl/datasets/0004/46467...,"{0: [0 - Loan_Number (string)], 1: [1 - Loan_T...",None,False,None,None,None,https://www.openml.org/d/46467
5812,None,46468,Give-Me-Some-Credit-Sampled,1,Originial Dataset Give-Me-Some-Credit 45577,pickle,arff,M. Freurer,None,None,...,None,None,https://openml1.win.tue.nl/datasets/0004/46468...,"{0: [0 - SeriousDlqin2yrs (nominal)], 1: [1 - ...",None,False,None,None,None,https://www.openml.org/d/46468


In [7]:
# from augmenters import (
#     TagAugmenter,
#     FeatureAugmenter, 
#     DatasetAugmenter,
#     SimilarityAugmenter,
#     NameAugmenter,
#     ScrapyAugmenter
# )

# description_column = 'description'
# dataset_id_column = 'dataset_id'
# tag_column = 'tag'
# features_column = '_features'
# name_column = 'name'
augmented_column = 'augmented_description'
# similar_datasets_column = 'similar_datasets'
# scraped_column = 'scraped_data'
# prompt_description_column = 'prompt_description'

# augmenters = [
#     TagAugmenter(description_column, tag_column, augmented_column, dataset_id_column, json_file_path="./GPT_semantic_tags.json"),
#     NameAugmenter(description_column, name_column, augmented_column),
#     FeatureAugmenter(description_column, features_column, augmented_column, max_features=100, reduce_features=True),
#     ScrapyAugmenter(description_column, scraped_column, augmented_column),
#     SimilarityAugmenter(description_column, augmented_column, similar_datasets_column),
# ]

# dataset_augmenter = DatasetAugmenter(augmenters=augmenters)
# augmented_df = dataset_augmenter.augment(df.copy())
# augmented_df

### Save augmented descriptions

In [8]:
# augmented_df[augmented_column].to_csv("augmented_description_checkpoint.csv", index=False)
# data = augmented_df[augmented_column].tolist()
data = pd.read_csv("augmented_description_checkpoint.csv")[augmented_column].tolist()

# Calculate embeddings

In [9]:
# embedding_model = SentenceTransformer("Salesforce/SFR-Embedding-2_R")

Loading checkpoint shards: 100%|██████████| 3/3 [00:07<00:00,  2.42s/it]


In [10]:
# embeddings = embedding_model.encode(data, show_progress_bar=True, batch_size=1)
# np.save('embeddings_checkpoint.npy', embeddings)

Batches: 100%|██████████| 5332/5332 [7:14:30<00:00,  4.89s/it]   


In [ ]:
# load embeddings
embeddings = np.load('embeddings_checkpoint.npy')

# Train topic model

In [ ]:
# Based on the below information, extract and synthesize human-readable tags/keywords/themes from the text, capitalized first letters of words. What is the main human-readable theme or subject matter discussed in the provided texts? What is the overarching, high-level theme of the texts, e.g. "Music", "Sports", "Environment", etc.? Please provide overarching themes that tie the different pieces of information together. What is/are the overarching, highest level theme(s) that you could use as a keyword(s)? Prefer single word tags/keywords, e.g. "Tennis" rather than "Tennis Match", "Prison" rather than "Prison Time", etc., however, if the term makes sense only as a whole, e.g. "Republic of the Congo", "COVID-19", then use it! Consider adding synonyms as well, e.g. for "Influenza", add "Flu", for "Car", add "Automobile", etc.! Some examples of human-readable themes are   "Agriculture", "Astronomy", "Chemistry", "Computational Universe", "Computer Systems", "Climate and Environment", "Culture", "Demographics", "Earth Science", "Economics", "Education", "Engineering", "Finance", "Geography", "Government", "Games", "Health", "History", "Human Activities", "Images", "Language", "Law", "Life Science", "Machine Learning", "Manufacturing", "Mathematics", "Medicine", "Meteorology", "Physical Sciences", "Politics", "Social Media", "Sociology", "Statistics", "Text & Literature",  "Transportation". Avoid tags/keywords that are too specific, e.g. "Serine Threonine Protein Kinase". Good theme examples are: "Birds", "Species Migration", "Air Pollution", or "War", "Government", "International Relations", "Politics". And be concise in theme generation, e.g. instead of "Income Prediction", say "Income", instead of "Demographic Information", say "Demographics"! Another important rule to obey - place more focus on the dataset names for theme extraction, if they exist. Say {"topic": {"themes": ["example1", "example2", ...], "overarching_themes": ["example13", "...", ...]}, "texts": [{"example1": 0.93, "example2": 0.68, "...": ..., ..., "example13": ..., ... (strings corresponding to themes and overarching themes for text 1, all of them)}], {"example1": 0.87, "example2": 0.34, "...": ..., ..., "example13": ..., ... (strings corresponding to themes and overarching themes for text 2, all of them)}]} and give your answer in JSON format, where under "topic" you put all themes and overarching themes, and in "texts", you give a confidence score in each theme and overarching theme for each text. "themes" and "overarching_themes" shouldn't overlap. If a theme is overarching and common to all texts, it should be present in all texts with a high score. Give only the ones with highest scores.
# For example, for this text:
# Text 1: Name: Biden Administration

# The Biden administration is preparing to roll out a sweeping border executive action as early as Tuesday, according to two sources familiar with the discussions, who cautioned that timing is fluid.

# White House officials have begun reaching out to mayors who represent cities along the US southern border to potentially join President Joe Biden when he announces the order, two other sources familiar with those conversations said.

# For weeks, administration officials have been working through an executive action that would dramatically limit migrants’ ability to seek asylum at the US southern border — part of a strategy to try to give Biden the upper hand on one of his Republican rival’s key campaign issues. The action is designed to potentially blunt Republican attacks on border security and preempt former President Donald Trump ahead of the first presidential debate, which will be held on June 27 on CNN.
# ---
# Text 2: Name: Trump conviction

# Now that a New York jury has convicted former President Donald Trump of all 34 felony charges of falsifying business records, the next obvious question is: Can a convicted felon run for president?

# Definitely.

# Trump meets all three requirements. There is, arguably, another criterion laid out in the 14th Amendment, where it states that no one who has previously taken an oath of office who engages in insurrection can be an officer of the US. But the US Supreme Court ruled earlier this year that Congress would have to pass a special law invoking this prohibition. That’s not happening any time soon.

# Judge Juan Merchan has scheduled Trump’s sentencing for July 11, which happens to be four days before the start of the Republican National Convention that is scheduled to take place in Milwaukee.

# It is technically possible, although perhaps unlikely for a first-time offender, that Trump could be sentenced to prison time.
# ---
# Text 3: Trump has vowed to give green cards to college grads. Could that actually happen?

# The candidate known for touting immigration crackdowns told a group of tech investors that he wanted to help foreign students stay in the US.

# “What I want to do, and what I will do, is — you graduate from a college, I think you should get automatically, as part of your diploma, a green card to be able to stay in this country,” Trump said during a June interview with “The All-In Podcast.”

# If the president-elect pursues this proposal after he takes office, and if Congress passes the legislation that would be required to enact it, the policy could pave the way for potentially millions of international students to become legal permanent residents.
# ---
# This would be your answer:
# {
#   "topic": {
#     "themes": [
#       "Biden Administration",
#       "Border",
#       "Executive Action",
#       "Asylum",
#       "Immigration",
#       "Trump",
#       "Felony",
#       "Business Records",
#       "Presidential Campaign",
#       "Republican",
#       "Debate",
#       "Former President",
#       "Conviction",
#       "Sentencing",
#       "Prison",
#       "14th Amendment",
#       "Insurrection",
#       "Supreme Court",
#       "Republican National Convention",
#       "College",
#       "Green Card",
#       "Legislation",
#       "Student"
#     ],
#     "overarching_themes": [
#       "Politics",
#       "Government",
#       "Law",
#       "Justice",
#       "Elections",
#       "Education"
#     ]
#   },
#   "texts": { 
#     "text1": {
#       "Biden Administration": 0.96,
#       "Border": 0.92,
#       "Executive Action": 0.91,
#       "Asylum": 0.88,
#       "Immigration": 0.84,
#       "Presidential Campaign": 0.82,
#       "Republican": 0.82,
#       "Debate": 0.78,
#       "Politics": 0.99,
#       "Government": 0.93,
#       "Law": 0.85,
#       "Elections": 0.72,
#     },
#     "text2": {
#       "Trump": 0.95,
#       "Felony": 0.92,
#       "Business Records": 0.97,
#       "Presidential Campaign": 0.84,
#       "Republican": 0.82,
#       "Former President": 0.98,
#       "Conviction": 0.92,
#       "Sentencing": 0.91,
#       "Prison": 0.85,
#       "14th Amendment": 0.82,
#       "Insurrection": 0.80,
#       "Supreme Court": 0.78,
#       "Republican National Convention": 0.76,
#       "Politics": 0.92,
#       "Government": 0.92,
#       "Law": 0.90,
#       "Justice": 0.88,
#       "Elections": 0.85,
#     },
#     "text3": {
#       "Immigration": 0.67,
#       "Trump": 0.98,
#       "Republican": 0.59,
#       "College": 0.98,
#       "Green Card": 0.93,
#       "Legislation": 0.89,
#       "Student": 0.89,
#       "Politics": 0.82,
#       "Government": 0.81,
#       "Law": 0.69,
#       "Education": 0.97
#     }
#   }
# }
# ---
# Now, the above was just an example. Now, do it for all the following text(s), generate many themes:
# Text 1: Name: COVID-19 Rio de Janeiro (City)\n\nTags: Context - World Health Organization (WHO) Coronavirus disease (COVID-19) is an infectious disease caused by a newly discovered coronavirus.\n\nThis dataset provides information on the number of confirmed cases, deaths, and recoveries by neighborhood in the city of Rio de Janeiro, Brazil. It is essential to note that this is a time-series data, and therefore, the number of cases on any given day represents a cumulative total. To obtain the number of new cases, one can calculate the difference between the current and previous days' data.\n\nThe data is available from April to December, providing a comprehensive overview of the pandemic's progression in the city. The dataset is a valuable resource for analyzing changes in the number of confirmed cases, deaths, and recoveries by neighborhood over time, as well as at the city level. It also enables the study of the spread of the disease in the city.\n\nDate, Hour, Neighborhood, Cases, Deaths, Recovered
# ---
# Text 2: Name: Covid-19 Turkey Daily Details Dataset\nTags: Context Turkey Covid 19 Dataset\n\nThis dataset has been created in accordance with the data shared by the Ministry of Health of the Republic of Turkey. The data is collected from the website of the Ministry of Health on a daily basis using data mining methods. The dataset is updated daily and shared on GitHub.\n\nThe dataset contains the following columns:\n\nTotal Tests: the cumulative number of tests carried out up to a specific date\nTotal Cases: the cumulative number of cases announced up to a specific date\nTotal Deaths: the cumulative number of deaths announced up to a specific date\nTotal Intensive Care: the cumulative number of people in intensive care announced until a specific date\nTotal Intubated: the cumulative number of intubated people announced until a specific date\nTotal Recovered: the cumulative number of people who have recovered\nDaily Tests: the number of people tested on a specific day\nDaily Cases: the number of new cases announced on a specific day\nDaily Deaths: the number of new deaths announced on a specific day\nDaily Recovered: the number of people who have recovered on a specific day\n\nFeatures: date, total tests, total cases, total deaths, total intensive care, total intubated, total recovered, daily tests, daily cases, daily deaths
# ---
# Text 3: Name: COVID-19 World Vaccination Progress\nTags: Context\n\nData is collected daily from Our World in Data GitHub repository for COVID-19, merged and uploaded.\n\nThe data contains the following information: Country, which is the country for which the vaccination information is provided; Country ISO Code, which is the ISO code for the country; Date, which is the date for the data entry; for some of the dates, we have only the daily vaccinations, while for others, only the cumulative total; Total Number of Vaccinations, which is the absolute number of total immunizations in the country; Total Number of People Vaccinated, which is a person, depending on the immunization scheme, will receive one or more, typically two, vaccines; at a certain moment, the number of vaccinations might be larger than the number of people; Total Number of People Fully Vaccinated, which is the number of people that received the entire set of immunization according to the immunization scheme, typically two; at a certain moment in time, there might be a certain number of people that received one vaccine and another number, smaller, of people that received all vaccines in the scheme; Daily Vaccinations, which is for a certain data entry, the number of vaccination for that date and country; Total Vaccinations per Hundred, which is the ratio, in percent, between vaccination number and total population up to the date in the country; Total Number of People Vaccinated per Hundred, which is the ratio, in percent, between population immunized and total population up to the date in the country; Total Number of People Fully Vaccinated per Hundred, which is the ratio, in percent, between population fully immunized and total population up to the date in the country; Number of Vaccinations per Day, which is the number of daily vaccination for that day and country; Daily Vaccinations per Million, which is the ratio, in parts per million, between vaccination number and total population for the current date in the country; Vaccines Used in the Country, which is the total number of vaccines used in the country up to date; Source Name, which is the source of the information, such as national authority, international organization, local organization, etc.; Source Website, which is the website of the source of information.\n\nThis dataset allows users to track COVID-19 vaccination progress in the world, answering questions such as which country is using what vaccine, in which country the vaccination program is more advanced, where are more people vaccinated per day, and where are more people vaccinated in terms of percentage from the entire population
# ---
# Text 4: Name: COVID-19 U.S. Daily Cases
# Tags: Context - U.S. COVID-19 Data
# This dataset provides daily updates on confirmed COVID-19 cases, hospitalizations, and deaths across the United States. It includes state-by-state breakdowns and national trends, allowing for a comprehensive analysis of how COVID-19 is affecting different regions. The data is updated regularly and helps public health officials and researchers to track the virus's spread, assess the effectiveness of interventions, and inform policy decisions.
# ---
# Text 5: Name: COVID-19 Global Testing Rates
# Tags: Context - Global Testing Data
# This dataset tracks the number of COVID-19 tests conducted worldwide, highlighting the importance of testing in managing the pandemic. It includes data on daily testing rates, cumulative tests by country, and comparisons of testing capacities among nations. The information is crucial for understanding how different countries are responding to the pandemic and for evaluating the effectiveness of their public health strategies.
# ---
# Text 6: Name: COVID-19 Variants Tracker
# Tags: Context - Virus Variants
# This dataset includes information on different COVID-19 variants identified globally, tracking their prevalence and mutations. It provides insights into how these variants affect transmissibility, severity, and vaccine efficacy. The data is essential for researchers and public health officials to monitor the evolution of the virus and make informed decisions regarding vaccination strategies and public health measures.
# ---
# Text 7: Name: COVID-19 Economic Impact Survey
# Tags: Context - Economic Effects of COVID-19
# This dataset captures the economic effects of the pandemic through surveys assessing job loss, business closures, and changes in income across various demographics. It provides a detailed look at how different sectors are affected and the long-term implications for the economy. This information is vital for policymakers and economists to design effective recovery strategies and support measures for affected individuals and businesses.
# ---
# Text 8: Name: COVID-19 Mental Health Impact Study
# Tags: Context - Mental Health
# This dataset examines the mental health effects of the COVID-19 pandemic, focusing on anxiety, depression, and overall well-being across different age groups and communities. It includes survey responses collected during various stages of the pandemic, providing valuable insights into the psychological toll of the crisis. Understanding these impacts is crucial for developing mental health resources and support systems in the wake of the pandemic.
# ---
# Text 9: Name: COVID-19 Vaccine Adverse Events Reporting
# Tags: Context - Vaccine Safety
# This dataset contains reports of adverse events following COVID-19 vaccinations, compiled from various health agencies and reporting systems. It includes information on the types of side effects reported, demographic data of affected individuals, and follow-up outcomes. This data is essential for monitoring vaccine safety and ensuring public confidence in vaccination programs as they play a crucial role in controlling the pandemic.

In [134]:
%load_ext autoreload
%autoreload 2

In [169]:
from sklearn.decomposition import IncrementalPCA
from river import stream, cluster
from bertopic.vectorizers import OnlineCountVectorizer
from sklearn.cluster import MiniBatchKMeans
from bertopic import BERTopic

class River:
    def __init__(self, model):
        self.model = model

    def partial_fit(self, embeddings):
        for embedding, _ in stream.iter_array(embeddings):
            self.model.learn_one(embedding)

        labels = []
        for embedding, _ in stream.iter_array(embeddings):
            label = self.model.predict_one(embedding)
            labels.append(label)

        self.labels_ = labels
        return self

umap_model = IncrementalPCA()
# cluster_model = River(cluster.DBSTREAM())
cluster_model = MiniBatchKMeans(n_clusters=500, random_state=0)

vectorizer_model = OnlineCountVectorizer(stop_words="english")
ctfidf_model = ClassTfidfTransformer(reduce_frequent_words=True, bm25_weighting=True)

model = BERTopic(
    umap_model=umap_model,
    hdbscan_model=cluster_model,
    vectorizer_model=vectorizer_model,
    ctfidf_model=ctfidf_model,
    # verbose=True
)

# documents is empty dataframe to which to append the documents
documents = pd.DataFrame()

for i in tqdm(range(0, len(data), 500), desc="Fitting data"):
    _, docs = model.partial_fit(data[i:i+500], embeddings[i:i+500].astype(np.float64))
    documents = pd.concat([documents, docs])

print(documents)

Fitting data:   9%|▉         | 1/11 [00:00<00:05,  1.78it/s]

                                              Document   ID  Topic
0    Name: anneal\n\nTags: study_1, study_14, study...    0    133
1    Name: kr-vs-kp\n\nTags: mythbusting_1, OpenML-...    1    219
2    Name: labor\n\nTags: mythbusting_1, study_1, s...    2    184
3    Name: arrhythmia\n\nTags: sport, study_1, stud...    3    247
4    Name: letter\n\nTags: AzurePilot, AzurePilot1,...    4    197
..                                                 ...  ...    ...
495  Name: fri_c4_250_25\n\nTags: artificial\n\n**A...  495    462
496  Name: fri_c3_500_50\n\nTags: artificial\n\n**A...  496    471
497  Name: fri_c3_500_10\n\nTags: artificial\n\n**A...  497    450
498  Name: fri_c1_250_10\n\nTags: artificial\n\n**A...  498    459
499  Name: fri_c1_250_50\n\nTags: artificial\n\n**A...  499    355

[500 rows x 3 columns]


Fitting data:  18%|█▊        | 2/11 [00:01<00:06,  1.43it/s]

                                              Document     ID  Topic
0    Name: fri_c0_500_5\n\nTags: \n\n**Author**:   ...    0.0  248.0
1    Name: fri_c0_500_50\n\nTags: artificial\n\n**A...    1.0  248.0
2    Name: fri_c0_100_25\n\nTags: artificial\n\n**A...    2.0  248.0
3    Name: fri_c4_100_100\n\nTags: artificial\n\n**...    3.0  248.0
4    Name: fri_c0_250_25\n\nTags: artificial\n\n**A...    4.0  248.0
..                                                 ...    ...    ...
891                                                     891.0  475.0
892                                                     892.0  485.0
893                                                     893.0  488.0
894                                                     894.0  493.0
895                                                     895.0  496.0

[896 rows x 3 columns]


Fitting data:  27%|██▋       | 3/11 [00:02<00:06,  1.24it/s]

...

                                              Document     ID  Topic
0    Name: Intersectional-Bias-Assessment-(Testing-...    0.0  458.0
1    Name: Reading_Hydro\n\nTags: \n\nData from the...    1.0  405.0
2    Name: Reading_Hydro_upstream\n\nTags: \n\nUpst...    2.0  405.0
3    Name: Reading_Hydro_upstream\n\nTags: \n\nUpst...    3.0  405.0
4    Name: Reading_Hydro_downstream\n\nTags: \n\nUp...    4.0  405.0
..                                                 ...    ...    ...
985                                                     985.0  495.0
986                                                     986.0  496.0
987                                                     987.0  497.0
988                                                     988.0  498.0
989                                                     989.0  499.0

[990 rows x 3 columns]


Fitting data:  91%|█████████ | 10/11 [00:08<00:00,  1.13it/s]

                                              Document     ID  Topic
0    Name: guillermo_seed_2_nrows_2000_nclasses_10_...    0.0  101.0
1    Name: guillermo_seed_3_nrows_2000_nclasses_10_...    1.0  101.0
2    Name: guillermo_seed_4_nrows_2000_nclasses_10_...    2.0  101.0
3    Name: riccardo_seed_0_nrows_2000_nclasses_10_n...    3.0  101.0
4    Name: riccardo_seed_1_nrows_2000_nclasses_10_n...    4.0  101.0
..                                                 ...    ...    ...
975                                                     975.0  494.0
976                                                     976.0  496.0
977                                                     977.0  497.0
978                                                     978.0  498.0
979                                                     979.0  499.0

[980 rows x 3 columns]


Fitting data: 100%|██████████| 11/11 [00:09<00:00,  1.18it/s]

                                              Document     ID  Topic
0    Name: test\n\nTags: \n\ntest\n\nFeatures: Prod...    0.0  290.0
1    Name: StudentsPerformance\n\nTags: \n\nThis da...    1.0  429.0
2    Name: DrinkDataset\n\nTags: \n\nfake dataset w...    2.0  429.0
3    Name: algae\n\nTags: \n\nThis dataset contains...    3.0  443.0
4    Name: Amphibian\n\nTags: \n\namphibians\n\nFea...    4.0  284.0
..                                                 ...    ...    ...
650                                                     650.0  492.0
651                                                     651.0  494.0
652                                                     652.0  497.0
653                                                     653.0  498.0
654                                                     654.0  499.0

[655 rows x 3 columns]
                                              Document     ID  Topic
0    Name: anneal\n\nTags: study_1, study_14, study...    0.0  133.0
1    Name:

In [206]:
documents.iloc[402]

Document    Name: quake\n\nTags: \n\n**Author**:   \n**Sou...
ID                                                      402.0
Topic                                                    68.0
Name: 402, dtype: object

In [117]:
doc_ids = range(len(data)) if data is not None else range(len([]))
documents = pd.DataFrame({"Document": data, "ID": doc_ids, "Topic": model.topics_, "Image": []})

repr_docs_mappings, _, _, _ = model._extract_representative_docs(
    model.c_tf_idf_, documents, model.topics_, 500, 4
)

ValueError: All arrays must be of the same length

In [ ]:
# test_umap_model = IncrementalPCA()
test_umap_model = UMAP(
    n_neighbors=3,
    n_components=5,
    min_dist=0.008,
)
test_embeddings = test_umap_model.fit_transform(embeddings)
# test_embeddings = test_umap_model.transform(embeddings)

In [74]:
test_cluster_model = River(cluster.DBSTREAM(
    clustering_threshold=0.1,  # Try small values like 0.1-0.3
    fading_factor=0.01,       # Small value for more stable clusters
    cleanup_interval=20,      # Larger value to prevent frequent cleanup
    intersection_factor=0.3,  # Default is good for separation
    minimum_weight=1.0        # Keep default unless you want to filter noise
)
)
# test_cluster_model.partial_fit(test_embeddings)

In [75]:
for i in tqdm(range(0, len(data), 5), desc="Fitting data"):
    test_cluster_model.partial_fit(embeddings[i:i+5])

Fitting data: 100%|██████████| 1067/1067 [04:29<00:00,  3.96it/s]


In [76]:
np.unique(test_cluster_model.labels_)

array([30, 31])

In [155]:
model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,0,2,0_stylizedmetaalbumpltdocstyextended_phen_chem...,"[stylizedmetaalbumpltdocstyextended, phen, che...",[Name: Stylized_Meta_Album_PLT_DOC_STY_Extende...
1,1,63,1_stock_pulsar_dmsnr_week,"[stock, pulsar, dmsnr, week, radio, open, cand...",NaN
2,2,1,2_futurebusiness_proceduralfairness_analcatdat...,"[futurebusiness, proceduralfairness, analcatda...",NaN
3,3,9,3_rabexxx_bertram_schattersternnyuedu_samprit,"[rabexxx, bertram, schattersternnyuedu, sampri...",NaN
4,4,1,4_bnglabornominal1000000_costoflivingadjustmen...,"[bnglabornominal1000000, costoflivingadjustmen...",NaN
...,...,...,...,...,...
495,495,107,495_default_vaccinations_vaccination_bioconcen...,"[default, vaccinations, vaccination, bioconcen...",[Name: AIDS_Virus_Infection_Prediction\n\nTags...
496,496,2,496_admin2_covariatex_fips_provincestate,"[admin2, covariatex, fips, provincestate, conf...",[Name: Covid19-us\n\nTags: \n\nDaily values of...
497,497,2,497_logpopulation_total2000_silver2000_bronze2000,"[logpopulation, total2000, silver2000, bronze2...",NaN
498,498,23,498_filannino_michele_dbworld_addresses,"[filannino, michele, dbworld, addresses, bagof...",NaN


In [89]:
model.get_representative_docs(1)

In [ ]:
# Only run once
vectorizer_model = CountVectorizer(stop_words="english", ngram_range=(1, 1))

try:
    representation_model = PartOfSpeech(model="en_core_web_lg")
    print("Model already installed")
except OSError:
    # If model isn't installed, download it
    print("Model not found, downloading...")
    import os
    os.system("python -m spacy download en_core_web_lg")


umap_model = UMAP(
    n_neighbors=3,
    n_components=5,
    min_dist=0.008,
)

hdbscan_model = HDBSCAN(
    min_cluster_size=3,
    metric="euclidean",
    cluster_selection_method="eom",
    prediction_data=True,
)

ctfidf_model = ClassTfidfTransformer(reduce_frequent_words=True)

model = BERTopic(
    verbose=True,
    min_topic_size=2,
    ctfidf_model=ctfidf_model,
    calculate_probabilities=True,
    umap_model=umap_model,
    # embedding_model=embedding_model, # don't pass embedding model, just pass embeddings in the next step (pre-computed)
    vectorizer_model=vectorizer_model,
    representation_model=representation_model,
)

topics, probs = model.fit_transform(data, embeddings)
np.save('topics_checkpoint.npy', topics)
np.save('probs_checkpoint.npy', probs)

2024-12-06 13:43:56,280 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm


Model already installed


OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
2024-12-06 13:44:07,573 - BERTopic - Dimensionality - Completed ✓
2024-12-06 13:44:07,574 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-12-06 13:46:31,604 - BERTopic - Cluster - Completed ✓
2024-12-06 13:46:31,623 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-12-06 13:48:31,629 - BERTopic - Representation - Completed ✓


In [ ]:
len(np.unique(topics))

957

In [18]:
model.save("model", serialization="safetensors", save_ctfidf=True)

In [ ]:
# model = BERTopic.load("model")

In [ ]:
# topics = np.load('topics.npy')
# probs = np.load('probs.npy')

In [ ]:
prompt = '''Based on the below information, extract and synthesize human-readable tags/keywords/themes from the text, capitalized first letters of words. What is the main human-readable theme or subject matter discussed in the provided texts? What is the overarching, high-level theme of the texts, e.g. "Music", "Sports", "Environment", etc.? Please provide overarching themes that tie the different pieces of information together. What is/are the overarching, highest level theme(s) that you could use as a keyword(s)? Prefer single word tags/keywords, e.g. "Tennis" rather than "Tennis Match", "Prison" rather than "Prison Time", etc.! Some examples of human-readable themes are   "Agriculture", "Astronomy", "Chemistry", "Computational Universe", "Computer Systems", "Climate and Environment", "Culture", "Demographics", "Earth Science", "Economics", "Education", "Engineering", "Finance", "Geography", "Government", "Games", "Health", "History", "Human Activities", "Images", "Language", "Law", "Life Science", "Machine Learning", "Manufacturing", "Mathematics", "Medicine", "Meteorology", "Physical Sciences", "Politics", "Social Media", "Sociology", "Statistics", "Text & Literature",  "Transportation". Also, don't give very similar tags/keywords, e.g. "Wine" and "Red Wine", just give one or the other in these cases. Avoid tags/keywords that are too specific, e.g. "Serine Threonine Protein Kinase". Good theme examples are: "Birds", "Species Migration", "Air Pollution", or "War", "Government", "International Relations", "Politics". Another important rule to obey - place more focus on the dataset names for theme extraction. And be concise in theme generation, e.g. instead of "Income Prediction", say "Income", instead of "Demographic Information", say "Demographics"! Also, extract the theme of the text, what it is about, instead of the type of problem it is, for instance we don't care about "Regression", "Numerical Features", "Data Analysis", "Data", "Outliers", "Subsampling" or things of that sort, but we care about the ESSENCE of the text! Say {"themes": [...], "overarching_themes": [...]} and give your answer in JSON format.
For example, for this text:
Text 1: The Biden administration is preparing to roll out a sweeping border executive action as early as Tuesday, according to two sources familiar with the discussions, who cautioned that timing is fluid.

White House officials have begun reaching out to mayors who represent cities along the US southern border to potentially join President Joe Biden when he announces the order, two other sources familiar with those conversations said.

For weeks, administration officials have been working through an executive action that would dramatically limit migrants’ ability to seek asylum at the US southern border — part of a strategy to try to give Biden the upper hand on one of his Republican rival’s key campaign issues. The action is designed to potentially blunt Republican attacks on border security and preempt former President Donald Trump ahead of the first presidential debate, which will be held on June 27 on CNN.
---
Text 2: Now that a New York jury has convicted former President Donald Trump of all 34 felony charges of falsifying business records, the next obvious question is: Can a convicted felon run for president?

Definitely.

Trump meets all three requirements. There is, arguably, another criterion laid out in the 14th Amendment, where it states that no one who has previously taken an oath of office who engages in insurrection can be an officer of the US. But the US Supreme Court ruled earlier this year that Congress would have to pass a special law invoking this prohibition. That’s not happening any time soon.

Judge Juan Merchan has scheduled Trump’s sentencing for July 11, which happens to be four days before the start of the Republican National Convention that is scheduled to take place in Milwaukee.
a
It is technically possible, although perhaps unlikely for a first-time offender, that Trump could be sentenced to prison time.
---
This would be your answer:
{"themes": ["Biden Administration", "Border", "Executive Action", "Asylum", "Immigration", "Trump", "Felony", "Business Records", "Presidential Campaign", "Republican", "Debate", "Former President", "Conviction", "Sentencing", "Prison", "14th Amendment", "Insurrection", "Supreme Court", "Republican National Convention"], "overarching_themes": ["Politics", "Government", "Law", "Justice", "Elections"]}
---
Now, the above was just an example. Now, do it for the following text(s), be concise!:
[DOCUMENTS]
The topic is described by the following keywords: [KEYWORDS]
'''

In [88]:
# Only run once
new_topics = model.reduce_outliers(data, topics, probabilities=probs, strategy="probabilities")
model.update_topics(data, topics=new_topics)

2024-10-06 01:26:50,709 - BERTopic - WARNING: Using a custom list of topic assignments may lead to errors if topic reduction techniques are used afterwards. Make sure that manually assigning topics is the last step in the pipeline.Note that topic embeddings will also be created through weightedc-TF-IDF embeddings instead of centroid embeddings.


In [90]:
# Only run once
# from extensions import MistralAI
from src.bertopic_extensions import MistralAI
from mistralai import Mistral

client = Mistral(api_key=os.getenv("MISTRAL_API_KEY"))

llm = MistralAI(
    client,
    prompt=prompt,
    model="mistral-large-latest",
    nr_docs=100,
    max_tokens = 8192,
    cache=False,
    generator_kwargs={
        "response_format": {
            "type": "json_object"
        },
    },
)

model.update_topics(data, topics=new_topics, representation_model=llm)

In [ ]:
# model.save("model_fine_tuned", serialization="safetensors", save_ctfidf=True)

In [35]:
# # Only run this step if the topic_representations aren't updated for some reason (error, etc.)
# import ast
# import json

# with open("../src/bertopic_extensions/_cache/mistral.json") as f:
#     mistral = json.load(f)

# for i, topic_data in enumerate(mistral):
#     themes = [(theme, 1) for theme in topic_data['Themes']]
#     overarching_themes = [(theme, 2) for theme in topic_data['Overarching themes']]
#     model.topic_representations_[i] = themes + overarching_themes

# Zeroshot step

In [ ]:
# Only run once
top_k = 2
top_k_topics = []
for i in range(len(data)):
    top_k_indices = np.argsort(probs[i])[::-1][:top_k]
    top_k_indices = [index for index in top_k_indices]
    top_k_indices = [index for index in top_k_indices if index != -1]
    top_k_topics.append([str(topic) for topic in top_k_indices])

model.get_topic_info(int(top_k_topics[0][0]))

In [ ]:
# Only run once
import json

with open('model_fine_tuned/topics.json') as f:
    topics_json = json.load(f)

topics_json

In [99]:
# Only run once
def split_themes(data):
    themes_dict = {}
    overarching_themes_dict = {}
    count = 0
    for topic, representations in data['topic_representations'].items():
        themes = []
        overarching_themes = []

        for rep in representations:
            themes.extend([theme for theme in json.loads(rep[0])['Themes']])
            overarching_themes.extend([theme for theme in json.loads(rep[0])['Overarching themes']])
        
        themes = list(dict.fromkeys(themes))
        overarching_themes = list(dict.fromkeys(overarching_themes))
        
        themes_dict[topic] = ', '.join(themes)
        overarching_themes_dict[topic] = ', '.join(overarching_themes)
        count+=1
    
    return themes_dict, overarching_themes_dict

themes_dict, overarching_themes_dict = split_themes(topics_json)

print("Themes Dictionary:")
for k, v in themes_dict.items():
    print(f"{k}: {v}")

print("\nOverarching Themes Dictionary:")
for k, v in overarching_themes_dict.items():
    print(f"{k}: {v}")

Themes Dictionary:
0: Hockey, Teams, Players, Games, Goals, Playoffs, Penalties, Shots, Periods, Season, Stanley Cup, Fans, Trades, Injuries, Captains, Coaches, Standings, Polls
1: Car, Cars, V6, Ford, Engine, Mustang, Probe, GT, V8, Gear, Auto, Nissan, Automatic, Airbag, Automobile, Maxima, Autos, Rotor, Opel, Strut, Manta, Buy, Manual, Safety
2: Guns, Militia, Firearms, Weapons, Nuclear, Government, Law, Politics, Crime, Safety, Constitution, Rights, Legislation
3: Cryptography, Privacy, Security, Encryption, Government, Policy, Data, Freedom, Rights, Organization, NSA, EFF, ACLU, NRA, TEMPEST, RSA, PGP, Export, Law, Constitution, Assembly, Arms, Symposium, Submissions, Systems, Files, Keys
4: Electronics, Circuits, Power Supply, Voltage, Current, Digital, Analog, Signal Processing, Measurement, Amplifiers, Components, Electrical Engineering, Data Acquisition, Sound, Noise
5: Clipper, Encryption, Algorithm, Government, Chip, Privacy, EXPN, NSA, Proposal, Sternlight, Denning, Csrcncsl

In [ ]:
# Only run once

def append_to_json(file_path, new_data):
    """Append new data to a JSON array stored in a file."""
    if os.path.exists(file_path):
        with open(file_path, 'r+') as file:
            # Load existing data
            try:
                data = json.load(file)
            except json.JSONDecodeError:
                data = []
            # Append new data
            data.append(new_data)
            # Move the file pointer to the beginning
            file.seek(0)
            # Write the updated data
            json.dump(data, file, indent=2)
            # Truncate the file to the current size
            file.truncate()
    else:
        with open(file_path, 'w') as file:
            # Write new data in a list
            json.dump([new_data], file, indent=2)
            
import requests

API_URL = os.getenv('HUGGINGFACE_DEDICATED_API_URL')
headers = {
	"Accept" : "application/json",
	"Authorization": f"Bearer {os.getenv('HUGGINGFACE_DEDICATED_API_KEY')}",
	"Content-Type": "application/json" 
}

def query(payload):
	response = requests.post(API_URL, headers=headers, json=payload)
	return response.json()

hypothesis_template = "This text is about: {}"

for i, topics in tqdm(enumerate(top_k_topics), total=len(top_k_topics), desc="Processing Topics"):
    topic_labels_list = []
    overarching_labels_list = []
    
    for topic_index in topics:
        # Using themes from the themes_dict
        labels = themes_dict[str(topic_index)]
        labels = labels.split(", ")
        topic_labels_list.append(labels)

        # Using overarching themes from the overarching_themes_dict
        overarching_labels = overarching_themes_dict[str(topic_index)]
        overarching_labels = overarching_labels.split(", ")
        overarching_labels_list.append(overarching_labels)
    
    # Flatten the list of lists and remove duplicates
    topic_list = [item for sublist in topic_labels_list for item in sublist]
    topic_list = list(set(topic_list))
    overarching_list = [item for sublist in overarching_labels_list for item in sublist]
    overarching_list = list(set(overarching_list))
    
    # Query the API instead of zeroshot_model
    output = query({
        "inputs": data[i],  # Assuming `data[i]` is the input text for the zeroshot classifier
        "parameters": {
            "candidate_labels": topic_list,
            "hypothesis_template": hypothesis_template,
            "multi_label": True
        }
    })
    
    append_to_json('tags.json', output)

    # Query for overarching labels
    output_overarching = query({
        "inputs": data[i],
        "parameters": {
            "candidate_labels": overarching_list,
            "hypothesis_template": hypothesis_template,
            "multi_label": True
        }
    })
    
    append_to_json('overarching_tags.json', output_overarching)
    
    # Print labels with scores > 0.7
    for label, score in zip(output['labels'], output['scores']):
        if score > 0.7:
            print(f"Label: {label}, Score: {score}")
    
    # Print overarching labels with scores > 0.5
    for label, score in zip(output_overarching['labels'], output_overarching['scores']):
        if score > 0.5:
            print(f"Overarching Label: {label}, Score: {score}")

### Same as above but model runs locally

In [ ]:
# Only run once
from transformers import pipeline
from transformers import AutoTokenizer

hypothesis_template = "This text is about: {}"
tokenizer = AutoTokenizer.from_pretrained("MoritzLaurer/deberta-v3-large-zeroshot-v2.0")
zeroshot_model = pipeline("zero-shot-classification", model="MoritzLaurer/deberta-v3-large-zeroshot-v2.0")

for i, topics in tqdm(enumerate(top_k_topics), total=len(top_k_topics), desc="Processing Topics"):
    topic_labels_list = []
    overarching_labels_list = []
    for topic_index in topics:
        # Using themes from the themes_dict
        labels = themes_dict[str(topic_index)]
        labels = labels.split(", ")
        topic_labels_list.append(labels)
        
        # Using overarching themes from the overarching_themes_dict
        overarching_labels = overarching_themes_dict[str(topic_index)]
        overarching_labels = overarching_labels.split(", ")
        overarching_labels_list.append(overarching_labels)
    
    # Flatten the list of lists and remove duplicates
    topic_list = [item for sublist in topic_labels_list for item in sublist]
    topic_list = list(set(topic_list))
    overarching_list = [item for sublist in overarching_labels_list for item in sublist]
    overarching_list = list(set(overarching_list))
    
    # Apply the zeroshot model
    output = zeroshot_model(data[i], topic_list, hypothesis_template=hypothesis_template, multi_label=True)
    append_to_json('results.json', output)
    
    output_overarching = zeroshot_model(data[i], overarching_list, hypothesis_template=hypothesis_template, multi_label=True)
    append_to_json('overarching_results.json', output_overarching)
    
    # Print labels with scores > 0.7
    for label, score in zip(output['labels'], output['scores']):
        if score > 0.7:
            print(f"Label: {label}, Score: {score}")
            
    for label, score in zip(output_overarching['labels'], output_overarching['scores']):
        if score > 0.5:
            print(f"Overarching Label: {label}, Score: {score}")